In [1]:
import numpy as np
import tensorflow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [4]:
MODEL_PATH = '/saved_models/nlp_models'

In [5]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 36.2 MB/s 
     |████████████████████████████████| 1.4 MB 40.9 MB/s 
     |████████████████████████████████| 233 kB 68.5 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=d6fabd8de82a4d20168d479c26fb5179bdb05da85434cd4c24e6e022646435d8
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154421 sha256=936e1fb618daa72c26e3f3624b8acc6ebf1ce4cf9d43556d754d1489f4ac6619
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


###**Text preprocessing functions**

In [6]:
from __future__ import unicode_literals
from hazm import *
import re
import random
from string import punctuation

def text_preprocess(text):
  normalizer = Normalizer()
  text = normalizer.normalize(text)
  text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
  return text

def text_scramble(text):
  words = text.split()
  random.shuffle(words)
  return ' '.join(words)


###**Loading and processing Corpus 1**
(VOA FARSI 2003-08)

In [7]:
random.seed(42)
np.random.seed(42)

In [8]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 50000)

In [9]:
MAX_LEN = 200
sents_cleaned = [text_preprocess(s) for s in sents]
sents_ready = filter(lambda s: len(s) < MAX_LEN, sents_cleaned)
sents_data = list(sents_ready)
print(len(sents_data))
random.shuffle(sents_data)
sents_data = sents_data[:45000]  # make dataset size 45000 for ease of computation

45733


In [10]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(sents_data)
tokenizer.word_index['<PAD>'] = 0
last_idx = len(tokenizer.word_index) + 1
tokenizer.word_index['<SOS>'] = last_idx

In [11]:
temp = [len(each.split()) for each in sents_data]
print('maximum sentence length:', sorted(temp)[-1])

maximum sentence length: 45


### **building the dataset**

In [12]:
def create_shuffles(sent_list, m):
  """
  this function creates m random shuffles of the sentence
  """
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, m):
      comb_set.add(text_scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [13]:
shuffle_pairs = create_shuffles(sents_data, 5)

In [14]:
shuffle_pairs[1][0], shuffle_pairs[1][1]

(' تاکنون صدها سرباز از زمانی که شبه نظامیان طرفدار طالبان کنترل شهر\u200cهای کوچک و قصبات این ناحیه را بدست گرفتند با آنها در حال زدوخورد بوده\u200cاند',
 {'از حال ناحیه در صدها این گرفتند زمانی شهر\u200cهای آنها و کنترل طرفدار سرباز نظامیان کوچک را تاکنون بوده\u200cاند شبه با که قصبات طالبان بدست زدوخورد',
  'بدست از ناحیه شهر\u200cهای زدوخورد که سرباز در شبه تاکنون زمانی آنها کنترل حال را و گرفتند بوده\u200cاند طالبان نظامیان قصبات طرفدار با کوچک این صدها',
  'بوده\u200cاند در سرباز کنترل تاکنون صدها که و زمانی بدست شهر\u200cهای این حال شبه را طرفدار ناحیه کوچک قصبات طالبان با از زدوخورد آنها نظامیان گرفتند',
  'زمانی قصبات حال طرفدار شهر\u200cهای که بدست شبه و گرفتند نظامیان این را با آنها تاکنون کنترل کوچک طالبان در سرباز از ناحیه بوده\u200cاند صدها زدوخورد',
  'و گرفتند ناحیه شبه حال زدوخورد این قصبات از بدست با بوده\u200cاند کوچک سرباز صدها نظامیان طرفدار را زمانی شهر\u200cهای که تاکنون آنها در کنترل طالبان'})

In [15]:
def create_dataframe(pairs):
  """
  creates a dataframe with two columns of original sentence and it's shuffles
  """
  original = []
  shuffled = []
  for pair in pairs:
    for p in pair[1]:
      original.append(pair[0])
      shuffled.append(p)

  df_dict = { 'Shuffled': shuffled, 'Original': original}
  df = pd.DataFrame(df_dict)
  return df


In [39]:
df = create_dataframe(shuffle_pairs)
df = df.sample(frac=1).reset_index(drop=True)
df

In [17]:
def create_dataset(df, train_percent=80):
  data_size = df.shape[0]
  count = math.floor(data_size * (train_percent/100))

  dataset = df['Shuffled'].values
  labels  = df['Original'].values

  train_data   = dataset[:count]
  train_labels = labels[:count]
  test_data   = dataset[count:]
  test_labels = labels[count:]

  return ((train_data, train_labels), (test_data, test_labels))




In [18]:
train_set, test_set = create_dataset(df)
X_train_text, Y_train_text = train_set
X_test_text, Y_test_text = test_set

###**making dataset ready for training**

In [19]:
sos_index = last_idx
def shift_output_sequence(seq_list):
  for seq in seq_list:
    seq.insert(0, sos_index)
  return seq_list


In [20]:
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
Y_train_seq = tokenizer.texts_to_sequences(Y_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)
Y_test_seq  = tokenizer.texts_to_sequences(Y_test_text)

In [21]:
MAX_SEQ_LEN = 50  
X_train = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN)
Y_train = keras.preprocessing.sequence.pad_sequences(Y_train_seq, maxlen=MAX_SEQ_LEN)

X_test  = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN)
Y_test  = keras.preprocessing.sequence.pad_sequences(Y_test_seq, maxlen=MAX_SEQ_LEN)

X_train_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_train_seq), maxlen=MAX_SEQ_LEN)
X_test_decoder  = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_test_seq),  maxlen=MAX_SEQ_LEN)

In [40]:
X_train_decoder[:2], Y_train[:2]

(array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0, 38546,  2280,   267,  2086,  3820, 24148,   821,
           303,  1492,  2677,    58,     6,   161,    77,   606,    15,
             7,    10,  1948,   883,    94],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0, 38546,    24,
            17,     4,   615,    13,     2,   138,    85,   564,   309,
          2322,   867,   177,    28,   112,     4,    11,   995,     7,
             5,  2998,     2,   138,  2516]], dtype=int32),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,  

In [23]:
X_train[:1],Y_train[:1]

(array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,  1492,    15,   303,    77,  2086,     7,
          3820,  1948,    58,  2280,   606,   161,    10,     6,   821,
            94, 24148,  2677,   267,   883]], dtype=int32),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,  2280,   267,  2086,  3820, 24148,   821,
           303,  1492,  2677,    58,     6,   161,    77,   606,    15,
             7,    10,  1948,   883,    94]], dtype=int32))

In [24]:
VOCAB_SIZE = len(tokenizer.word_index)
VOCAB_SIZE

38546

###**Encoder-Decoder network**

In [25]:
num_samples = X_train.shape[0]

In [26]:
def custom_loss(y_actual,y_pred):
  y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.dtypes.int32)
  find_match =tf.reduce_prod(tf.transpose(y_actual)[...,None]- tf.abs(y_pred[None,...]), 0)
  find_idx = tf.equal(find_match,tf.zeros_like(find_match))
  return (tf.reduce_sum(tf.cast(find_idx, tf.float32)))/MAX_SEQ_LEN


In [28]:
encoder_embedding_size = 64
decoder_embedding_size = 64
lstm_units = 256

tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

encoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 1, output_dim=encoder_embedding_size, input_length=MAX_SEQ_LEN, mask_zero=True)(encoder_input)

_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, return_state=True)(encoder_embedding)

encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

decoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 2, output_dim=decoder_embedding_size, mask_zero=True)(decoder_input)

decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(decoder_embedding, initial_state=encoder_state)

middle_dense_1 = keras.layers.Dense(lstm_units, activation="relu")(decoder_lstm_output)

decoder_output = keras.layers.Dense(VOCAB_SIZE + 1, activation="softmax")(middle_dense_1)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

scce = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Nadam(learning_rate=0.003)

model.compile(loss=[scce, custom_loss], loss_weights=[1, 4/num_samples], optimizer=optimizer, metrics=["accuracy"])
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 64)     2467008     ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 64)     2467072     ['input_4[0][0]']                
                                                                                            

In [29]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=15, validation_split=0.15, batch_size=256)

Epoch 1/15
592/592 [==============================] - 451s 743ms/step - loss: 1.2998 - accuracy: 0.5745 - val_loss: 0.2761 - val_accuracy: 0.9161
Epoch 2/15
592/592 [==============================] - 435s 735ms/step - loss: 0.1589 - accuracy: 0.9467 - val_loss: 0.0972 - val_accuracy: 0.9636
Epoch 3/15
592/592 [==============================] - 432s 731ms/step - loss: 0.0571 - accuracy: 0.9746 - val_loss: 0.0458 - val_accuracy: 0.9788
Epoch 4/15
592/592 [==============================] - 433s 731ms/step - loss: 0.1322 - accuracy: 0.9657 - val_loss: 0.0283 - val_accuracy: 0.9875
Epoch 5/15
592/592 [==============================] - 431s 728ms/step - loss: 0.0118 - accuracy: 0.9943 - val_loss: 0.0149 - val_accuracy: 0.9936
Epoch 6/15
592/592 [==============================] - 431s 728ms/step - loss: 0.0051 - accuracy: 0.9976 - val_loss: 0.0102 - val_accuracy: 0.9960
Epoch 7/15
592/592 [==============================] - 432s 729ms/step - loss: 0.0043 - accuracy: 0.9980 - val_loss: 0.0091 -

**99.9% training accuracy**

In [45]:
model.save("sentence_model_best.h5")

**model evaluation on test data**

In [ ]:
model = keras.models.load_model("sentence_model_best.h5", custom_objects={"custom_loss":custom_loss})

In [31]:
ids = np.argmax(model.predict([X_test[:1], X_test_decoder[:1]]), axis=-1)
ids, Y_test[:1]

(array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,  1086,     4,    49,   361,    19,   252,  4955,
           293,   109,    53,   555,  1312,   136,     5,   253,    82,
         21150,     7,  2013,  4074,    14]]),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,  1086,     4,    49,   361,    19,   252,  4955,
           293,   109,    53,   555,  1312,   136,     5,   253,    82,
         21150,     7,  2013,  4074,    14]], dtype=int32))

In [42]:
print("Evaluate on test data")
results = model.evaluate([X_test, X_test_decoder], Y_test, batch_size=256)
print("test loss, test acc:", results)

Evaluate on test data
175/175 [==============================] - 49s 278ms/step - loss: 0.0059 - accuracy: 0.9984
test loss, test acc: [0.00588442524895072, 0.9983782768249512]


###**Rebuild sentences**

In [33]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

def remove_padding_and_join(texts):
  for i in range(len(texts)):
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts
    

def reconstruct_predicted_list(texts):
  for i in range(len(texts)):
    pad_start = texts[i].index('<PAD>')
    texts[i] = texts[i][pad_start:]
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts

def get_predictions_on_test(count):
    Y_preds = model.predict([X_test[:count], X_test_decoder[:count]])
    encoded_argmax  = np.argmax(Y_preds, axis=-1)
    return encoded_argmax, Y_test[:count]

def get_predicted_sentences(predicted):
  predicted_word_list = list(map(sequence_to_text, predicted))
  predicted_sent_list = reconstruct_predicted_list(predicted_word_list)
  return predicted_sent_list

def get_original_sentences(original):
  original_word_list  = list(map(sequence_to_text, original))
  original_sent_list = remove_padding_and_join(original_word_list)
  return original_sent_list 


def rebuild_sentence_from_testset(count=20):
  predicted, original = get_predictions_on_test(count)
  predicted_sent_list = get_predicted_sentences(predicted)
  original_sent_list  = get_original_sentences(original)
  return predicted_sent_list, original_sent_list
  


In [41]:
preds, origs = rebuild_sentence_from_testset()
preds, origs

(['دانشجویان و دیگر مرد ایران خواستار آزادی\u200cهای بیشتر بودند تا اینکه سرکوب مردم از سوی پلیس اعتراضها را عمدتا خاموش کرد',
  'دو موسسه عظیم مالی جهانی موافقت کردند تا تیمی برای مطالعه و جمع آوری اطلاعات لازم برای تخمین هزینه\u200cهای باز سازی عراق جنگ زده به آن کشور اعزام دارند',
  'وی امتناع کرد و دستگیر شد',
  'جهاد اسلامی تائید کرده است مردانی که کشته شدند عضو آن گروه بودند',
  'پرزیدنت بوش خواسته است به آنها شانس کسب شهر وندی آمریکا داده شود در حالی که برخی از اعضای حزب جمهوریخواه می\u200cگویند چنین کاری به معنای بخشودگی خواهد بود',
  'روز سه شنبه رهبران کشور\u200cهای ۷ گانه بزرگ جهان پشتیبانی خود را از طرح آمریکا بنام نقشه راهنمائی صلح خاورمیانه اعلام کردند و در مورد آینده روشن بهبودی اقتصاد جهان اظهار اطمینان کردند',
  'در واشنگتن سخنگوی کاخ سفید بار دیگر موضع آمریکا را مورد تاکید قرارداد و اظهار داشت عملیات ترکیه در شمال عراق باید کوتاه مدت و کاملا در محدوده هدف\u200cهای خود باشد',
  'در این میان در ولایت اوروزگان در مرکز افغانستان سربازان افغان با پشتیبانی نیروهای ائتلاف شش

In [35]:
path = 'data/test_sents.txt'
with open(path, 'r', encoding='utf-8') as f:
  test_sents = [line.strip() for line in f.readlines()]

MAX_LEN = 190
test_sents_cleaned = [text_preprocess(s) for s in test_sents]
len(test_sents_cleaned)

50

In [36]:
test_df = create_dataframe(create_shuffles(test_sents_cleaned, 2))
test_df

,Shuffled,Original
0,امروزه که کنیم شده سفر است آسوده‌تر اپلیکیشن س...,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر س...
1,کنیم که سفر اپلیکیشن امروزه سفر باعث آسوده‌تر ...,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر س...
2,دنیای معنی حال در به این دست واژه شکل‌گیری افر...,معنی و مصادیق این واژه در حال حاضر در دنیای فن...
3,در آینده‌نگر فناوری و مصادیق حال در افراد جاه‌...,معنی و مصادیق این واژه در حال حاضر در دنیای فن...
4,در بیشتر کودکان را خود امروزه می‌گذرانند وقت م...,امروزه کودکان بیشتر وقت خود را در فضای مجازی م...
...,...,...
95,مشکل قضائی محکوم خبرنگاران را کنید رفع و بازداشتی,مشکل قضائی خبرنگاران محکوم و بازداشتی را رفع کنید
96,رایگان در و فوری ویدیویی روز و شماست تازه‌ترین...,تازه‌ترین خبرهای روز و اخبار فوری به‌صورت ویدی...
97,ویدیویی شماست رایگان فوری روز و در خبرهای و به...,تازه‌ترین خبرهای روز و اخبار فوری به‌صورت ویدی...
98,کنترل رسانه‌هایی می‌دانید چه را کسی می‌کند آنه...,رسانه‌هایی که همه می‌دانید چه کسی آنها را کنتر...


In [43]:
outer_sents = test_df['Shuffled'].values
label_sents = test_df['Original'].values

outer_seq = tokenizer.texts_to_sequences(outer_sents)
label_seq = tokenizer.texts_to_sequences(label_sents)

X_outer  = keras.preprocessing.sequence.pad_sequences(outer_seq, maxlen=MAX_SEQ_LEN)
X_outer_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(label_seq), maxlen=MAX_SEQ_LEN)

Y_outer_preds = model.predict([X_outer, X_outer_decoder])
outer_results  = np.argmax(Y_outer_preds, axis=-1)

outer_preds_sents = get_predicted_sentences(outer_results)
result_df = pd.DataFrame({'predictions': outer_preds_sents, 'Original': label_sents})

In [44]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
result_df

,predictions,Original
0,کیب سفر باعث شده است که امروزه نسلمانان سفر کنیم,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر سفر کنیم
1,کیب سفر باعث شده است که امروزه نسلمانان سفر کنیم,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر سفر کنیم
2,معنی و ترف این واژه در حال حاضر در دنیای فناوری به دست افراد ٢٠٨ و ترف در حال دماوند است,معنی و مصادیق این واژه در حال حاضر در دنیای فناوری به دست افراد جاه‌طلب و آینده‌نگر در حال شکل‌گیری است
3,معنی و ترف این واژه در حال حاضر در دنیای فناوری به دست افراد ٢٠٨ و ترف در حال دماوند است,معنی و مصادیق این واژه در حال حاضر در دنیای فناوری به دست افراد جاه‌طلب و آینده‌نگر در حال شکل‌گیری است
4,امروزه کودکان بیشتر وقت خود را در فضای لویانک می‌گذرانند,امروزه کودکان بیشتر وقت خود را در فضای مجازی می‌گذرانند
5,امروزه کودکان بیشتر وقت خود را در فضای لویانک می‌گذرانند,امروزه کودکان بیشتر وقت خود را در فضای مجازی می‌گذرانند
6,استفاده از امضای ایمیل کارها را ساده‌تر می‌کند,استفاده از امضای ایمیل کارها را ساده‌تر می‌کند
7,استفاده از امضای ایمیل کارها را ساده‌تر می‌کند,استفاده از امضای ایمیل کارها را ساده‌تر می‌کند
8,روزهای پایانی سال است و خیلی از ما در هرگیسیا نوشتن اهداف و ترف سال جدید هستیم,روزهای پایانی سال است و خیلی از ما در تکاپوی نوشتن اهداف و آرزوهای سال جدید هستیم
9,روزهای پایانی سال است و خیلی از ما در هرگیسیا نوشتن اهداف و ترف سال جدید هستیم,روزهای پایانی سال است و خیلی از ما در تکاپوی نوشتن اهداف و آرزوهای سال جدید هستیم
